<a href="https://colab.research.google.com/github/ishgirwan/omdena_hdi/blob/master/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  29234      0 --:--:-- --:--:-- --:--:-- 29402
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
     |████████████████████████████████| 61kB 2.8MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
\ [1 files][114.8 MiB/114.8 MiB]                                                
Operation completed over 1 objects/114.8 MiB.                                    
Copying gs://

In [3]:
!pip install rasterio
#pip install pytorch-lightning
!pip install pytorch-lightning-bolts
#!pip install albumentations
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

     |████████████████████████████████| 18.1MB 1.3MB/s 
     |████████████████████████████████| 143kB 3.4MB/s 
     |████████████████████████████████| 481kB 10.4MB/s 
     |████████████████████████████████| 276kB 17.5MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=36c5e11ed6df4d4bb41d4b12b2df9369730ec8f78017c969ef743282e6bc7beb
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
ERROR: pytorch-lightning 0.10.0 has requirement future>=0.17.1, but you'll have future 0.16.0 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/PytorchLightning/pytorch-lightning.git (to revision master) to /tmp/pip-req-build-g79ihh5j
  Running command git clone -q https://github.com/PytorchLightning/pytorch-l

In [4]:
import numpy as np
import pandas as pd
import rasterio

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.metrics import MeanAbsoluteError
#import albumentations as A

#from sklearn.preprocessing import MinMaxScaler    
from sklearn.metrics import r2_score

import glob
import os

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
csv_path = '/content/drive/My Drive/omdena/hdi_with_geometry.csv'
root_dir = '/content/drive/My Drive/Landsat/'

In [6]:
pip install efficientnet_pytorch


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=81900ba02a67a9534d97eed862384b31fe6bde1eae4ff553949ae91f4236ee17
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [7]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b3')
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [8]:
#model._conv_stem.in_channels = 6
model._conv_stem = nn.Conv2d(6, 40, kernel_size=3, stride=1, padding=3, bias=False)
#model._fc.out_features = 1
model._swish = nn.Sigmoid()
model

EfficientNet(
  (_conv_stem): Conv2d(6, 40, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
 

In [9]:
class MyDataset(Dataset):
    """
    Generate normalized, rescaled and transformed datasets
    """

    def __init__(self, dataset_type, transform=None):
        
        super().__init__()
        self.df = dataset_type
        self.transform = transform

    def __len__(self):
        
        return len(self.df)
    
    def __getitem__(self, idx):
       
        if torch.is_tensor(idx):
              idx = idx.tolist()

        # generate image sample
        image_path = self.df['image_path'].iloc[idx] 
        image_sample = rasterio.open(str(image_path), "r")
        bands = [i for i in range(1, image_sample.count+1)]
        image_sample = image_sample.read(bands)
        image_sample = image_sample.astype('float32')

        # generate hdi sample

        hdi_sample = self.df['HDI'].iloc[idx]

        # Normalize the image sample and rescale it between 0 and 1
        for ch in range(image_sample.shape[0]):
            # standardize
            image_sample[ch] = (image_sample[ch] - np.nanmean(image_sample[ch])) / np.nanstd(image_sample[ch])
            
            # normalize
            image_sample[ch] = (image_sample[ch] - np.nanmin(image_sample[ch])) / (np.nanmax(image_sample[ch]) - np.nanmin(image_sample[ch]))
        
        # convet nan to 0
        image_sample[np.isnan(image_sample)] = 0

        if self.transform:
            image_sample = self.transform(image_sample)

        #print('hdi_sample', hdi_sample)

        return [image_sample.permute(1, 0, 2),  hdi_sample.astype('float32')]
    

In [10]:
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor,  ModelCheckpoint

# default used by the Trainer
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=4,
    strict=False,
    verbose=True,
    mode='min')


# DEFAULTS used by the Trainer
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/My Drive/omdena/ckpt/model.ckpt',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
    prefix=''
)

lr_monitor = LearningRateMonitor(logging_interval='step')


In [11]:
def my_collate(batch):

    max_wh = 0

    for item in batch:
        image = item[0]
        w = image.shape[1]
        h = image.shape[2]
        max_i = np.max([w, h])
        if max_i > max_wh:
            max_wh = max_i
    
    #print(max_wh)

    data = []

    for item in batch:
        image = item[0]
        rows = image.shape[1]
        cols = image.shape[2]
        rows_diff = max_wh - rows
        cols_diff = max_wh - cols
        cols_half = int(cols_diff / 2)
        rows_half = int(rows_diff / 2)
        padding = (cols_half, cols_diff-cols_half, rows_half, rows_diff-rows_half)
        image_pad = F.pad(image, padding, 'constant', 0)
        #print(image.shape)
        #print(image_pad.shape)
        #print(type(image_pad))
        data.append(image_pad)

    target = [item[1] for item in batch]
    return [data, target]

In [12]:
class Model(pl.LightningModule):

    def __init__(self, model, batch_size=1, learning_rate=.001):
        super().__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters()
        self.model = model
        self.batch_size = batch_size

    def forward(self, x):
        x = self.model(x)
        #print('shape', x.shape)
        x = torch.mean(x, 1)
        return x

    def prepare_data(self):

        df = pd.read_csv(csv_path)
        df['image_path'] = root_dir + df['unique code'].astype(str) + '.tif' 
        df = df.sample(frac=1, random_state=1).reset_index(drop=True)
        # split the dataset
        train, validate, test = np.split(df, [int(.95*len(df)), int(.975*len(df))]) 

        # transforms
        train_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])

        validate_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])
        # create datasets for training, validation and test
        self.train_dataset = MyDataset(dataset_type=train, transform=train_transform)
        self.validate_dataset = MyDataset(dataset_type=validate, transform=validate_transform)
        self.test_dataset = MyDataset(dataset_type=test, transform=validate_transform) 
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, self.batch_size, shuffle=True, num_workers=4, collate_fn=my_collate, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.validate_dataset, self.batch_size, num_workers=4,collate_fn=my_collate, pin_memory=True) 

    def test_dataloader(self):
        return DataLoader(self.test_dataset, self.batch_size, num_workers=4, collate_fn=my_collate, pin_memory=True) 

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        #print(y, y_hat)
        loss = F.l1_loss(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        #print(y, y_hat)
        loss = F.l1_loss(y_hat, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_R-square', r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    #def validation_epoch_end(self, outputs):
    #   avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #  return {'val_loss': avg_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        loss = F.l1_loss(y_hat, y)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_R-square', r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()),  on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def backward(self, trainer, loss, optimizer, optimizer_idx):
        loss.backward()

    def optimizer_step(self, current_epoch, batch_idx, optimizer, 
      optimizer_idx, second_order_closure=None, 
       on_tpu=False, using_native_amp=False, using_lbfgs=False):
        optimizer.step()


In [13]:
# init model
model_one = Model(model, batch_size=8)
#from pytorch_lightning.core.memory import ModelSummary
#ModelSummary(model_one, mode='full')
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger('/content/drive/My Drive/omdena/tb_logs', name='my_model')

#train
root_path = '/content/drive/My Drive/omdena'

In [14]:
#seed
pl.seed_everything(1234)

trainer = pl.Trainer(tpu_cores=8, precision=16, logger=logger, checkpoint_callback=checkpoint_callback, progress_bar_refresh_rate=50, accumulate_grad_batches=2, fast_dev_run=False,\
                    default_root_dir=root_path, auto_lr_find=True, gradient_clip_val=0.5,\
                    profiler=True,  max_epochs=1000, callbacks=[lr_monitor, early_stop, PrintTableMetricsCallback()])

trainer.fit(model_one)

GPU available: False, used: False
TPU available: True, using: 8 TPU cores
Using native 16bit precision.
training on 8 TPU cores
INIT TPU local core: 0, global rank: 0 with XLA_USE_BF16=1
Exception in device=TPU:0: dictionary update sequence element #0 has length 1; 2 is required
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 330, in _mp_start_fn
    _start_fn(index, pf_cfg, fn, args)
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/xla_multiprocessing.py", line 324, in _start_fn
    fn(gindex, *args)
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/accelerators/tpu_backend.py", line 122, in tpu_train_in_process
    self.trainer.train_loop.setup_training(model)
  File "/usr/local/lib/python3.6/dist-packages/pytorch_lightning/trainer/training_loop.py", line 132, in setup_training
    self.trainer.logger.save()
  File "/usr/local/lib/python3.6/dist-packages/pytorch_li

Exception: ignored

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir='/content/drive/My Drive/omdena/tb_logs'

In [ ]:
# test
trainer.test()

In [ ]:
from sklearn.metrics import mean_squared_error
A = [0.4062, 0.2288, 0.4386, 0.4160]
B = [0.4140, 0.3354, 0.6566, 0.6081]

mse = mean_squared_error(A, B)
mse